<a href="https://colab.research.google.com/github/seunghee0518/R_File/blob/main/R_%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95_3%EC%9E%A5_%EB%8B%A4%EC%A4%91%EA%B3%B5%EC%84%A0%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<표 3.1> 해군병원의 인력자료
* 분산팽창인자값 구하기
* Y : 월간 의사 연 근무시간
* X1 : 일 평균 환자수
* X2 : 월간 X-ray 촬영 횟수
* X3 : 월간 이용 병석수
* X4 : 해당 지역의 병원 이용가능 인구/100
* X5 : 평균 입원일

In [ ]:
#데이터 불러오기
hospital.url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95/hospital.txt'
hospital = read.table(hospital.url, header=T)
head(hospital, 3)
attach(hospital)

각 변수의 대한 분산팽창인자 값 구하기

In [ ]:
#1. 회귀적합 결과
hospital.lm = lm(Y ~ ., data = hospital)
summary(hospital.lm)

In [ ]:
anova(hospital.lm)

* 추정회귀식 : Y = 1962.948-15.852 X1 +0.056 X2 + 1.590 X3 - 4.219 X4 - 394.314 X5
* 분산σ2의 추정량 MSE = 412277
* 결정계수 R2 = 0.9908
* 경험적으로 Y는 X1, X3, X5에 비례할 것으로 예상되나 이들의 설명변수에 대한 회귀계수의 추정값의 부호가 음이므로 설명변수들 간에 다중공선성의 존재가 예상되며 따라서 분산팽창인자 등에 대한 진단이 요구된다

In [ ]:
#분산팽창인자계산을 위한 패키지 : fmsb - VIF()
install.packages("fmsb")
library(fmsb)

In [ ]:
#분산팽창인자계산
vif1 = VIF(lm(X1 ~ X2+X3+X4+X5, data = hospital))
vif2 = VIF(lm(X2 ~ X1+X3+X4+X5, data = hospital))
vif3 = VIF(lm(X3 ~ X1+X2+X4+X5, data = hospital))
vif4 = VIF(lm(X4 ~ X1+X2+X3+X5, data = hospital))
vif5 = VIF(lm(X5 ~ X1+X2+X3+X4, data = hospital))

In [ ]:
#분산팽창인자 표
variable = c('X1', 'X2', 'X3', 'X4', 'X5')
vif.value = c(vif1, vif2 ,vif3, vif4, vif5)
vif = data.frame(variable, vif.value)
vif

* 최대분산팽창인자 : VIF1 = 9597.57
* 따라서 다중공선성 문제가 존재함

In [ ]:
#독립변수들의 상관계수
cor(hospital[, -6])

* 설명변수들 사이에 (특히 X1, X2, X3,X4) 강한 선형종속관계가 있음을 짐작 할 수 있음
* 이 중 X1, X3사이의 표본상관계수가 0.9999로 매우 강한 상관관계가 있다.



In [ ]:
# 설명변수 X1을 제거한 회귀적합
summary(lm(Y~X2+X3+X4+X5, data = hospital))

* X1을 제외한 회귀식 : Y = 2032.188 + 0.05608 X2 + 1.0884 X3 - 5.0041 X4 - 410.083 X5
* 결정계수 R2 = 0.99080 : 거의 변하지 않음
* 또한 각각의 추정된 회귀계수의 표준오차도 모두 줄어즘

In [ ]:
#X1을 제외한 분산팽창인자
exceptX1.vif2 = VIF(lm(X2 ~ X3+X4+X5, data = hospital))
exceptX1.vif3 = VIF(lm(X3 ~ X2+X4+X5, data = hospital))
exceptX1.vif4 = VIF(lm(X4 ~ X2+X3+X5, data = hospital))
exceptX1.vif5 = VIF(lm(X5 ~ X2+X3+X4, data = hospital))

In [ ]:
#X1을 제외한분산팽창인자 표(X1포함한 VIF함께 비교)
exceptX1.vif.value = c('NULL', exceptX1.vif2 ,exceptX1.vif3, exceptX1.vif4, exceptX1.vif5)
exceptX1.vif = data.frame(variable, vif.value, exceptX1.vif.value)
exceptX1.vif

* X1을 포함하는 경우와 비교했을때 모두 작은 값을 가짐

## 분산분석표 작성

In [ ]:
anova(market.lm)

* SS(X1) = 485.57 : X1이 적합됬을 때 제곱합
* SS(X2|X1) = 22.30 : X1이 적합된 후, 변수 X2가 추가되었을 때의 추가제곱합
* 회귀제곱합 SS(X1, X2) = SS(X1)+SS(X2|X1) = 507.87


# 표준화된 중회귀모형 적합
* lm.beta 패키지 이용

In [ ]:
install.packages("lm.beta")
library(lm.beta)

In [ ]:
market.beta = lm.beta(market.lm)
print(market.beta)

In [ ]:
coef(market.beta)

In [ ]:
summary(market.beta)

* 적합된 표준회귀계수 모형 : Y* = 0.7016Z1 + 0.3376Z2
* 이때 X1의 계수가 X2의 계수보다 크므로 상대적으로 X1의 영향이 더 큼을 알 수 있음음

###